# Script for realtime predictions with tensorflow an a self made dataset

## Import all nessesary dependencies with time check

In [1]:
# Import time for performance measurements
import time
import numpy as np

# Start time measuring
start_time = time.time()

# Import picamera
#import picamera
#import picamera.array
print("--- %s seconds for picamera imports ---" % (time.time() - start_time))

#Import tflearn libraries
from tflearn import DNN
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
print("--- %s seconds for tflearn imports ---" % (time.time() - start_time))


--- 6.008148193359375e-05 seconds for picamera imports ---
--- 2.0354011058807373 seconds for tflearn imports ---


## Create the convolutional neural network with realtime image preprocessing

In [2]:
# Start time measuring
start_time = time.time()

# Real-time data augmentation (This is only used while training the DNN)
img_aug = ImageAugmentation()
img_aug.add_random_blur(0.25)
img_aug.add_random_rotation(max_angle=10.0)

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()
print("--- %s seconds for image preprocessing functions ---" % (time.time() - start_time))

# Convolutional network building
network = input_data(shape=[None, 320, 240, 3],
                     data_augmentation=img_aug,
                    data_preprocessing=img_prep)
network = conv_2d(network, 16, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 64, activation='relu')
network = dropout(network, 1)
network = fully_connected(network, 7, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Create the Neural Network
model = DNN(network, tensorboard_verbose=1)
print("--- %s seconds for creating the neural network ---" % (time.time() - start_time))

# Load the model
model.load('models/final_model_aug_prep')
print("--- %s seconds for importing the model ---" % (time.time() - start_time))

--- 0.025870800018310547 seconds for image preprocessing functions ---
--- 1.745755910873413 seconds for creating the neural network ---
--- 2.711277961730957 seconds for importing the model ---


## Get camera images and transform them to numpy arrays for live feeding into our cnn

In [3]:
# Start time measuring
start_time = time.time()

# Capture images for prediction
with picamera.PiCamera() as camera:
    with picamera.array.PiRGBArray(camera) as output:
        camera.resolution = (640, 480)
        print("--- %s seconds for camera ready to use ---" % (time.time() - start_time))
        camera.capture(output, 'rgb')
        print("--- %s seconds for image captures and transformed ---" % (time.time() - start_time))
        print('Captured %dx%dx%d image' % (
                output.array.shape[1], output.array.shape[0], output.array.shape[2]))
        X = np.array([output.array.shape[1], output.array.shape[0], output.array.shape[2]])
        prediction_time = time.time()
        model.predict(X)
        print("--- %s seconds for the prediction ---" % (time.time() - prediction_time))
        print("--- %s seconds for making a prediction ---" % (time.time() - start_time))
        output.truncate(0)

NameError: name 'picamera' is not defined